In [1]:
from molar import ClientConfig, Client

admin_cfg = ClientConfig(server_url="http://localhost:8000",
                         email="radifar@protonmail.com",
                         password="radifar123",
                         database_name="main")

admin_client = Client(admin_cfg)

admin_client.test_token()

{'email': 'radifar@protonmail.com',
 'is_superuser': True,
 'is_active': True,
 'full_name': 'Muhammad Radifar',
 'created_on': '2021-10-24T06:22:05.802750',
 'user_id': 1}

***

I'd prefer viewing the entries in dictionary rather than pandas dataframe, so I overwrite the `view_entries` method from `Client` class.

In [2]:
class RewovenClient(Client):
    def __init__(self, cfg):
        super().__init__(cfg)
    
    def view_entries(
        self,
        database_name = None,
    ):
        database_name = database_name or self.cfg.database_name
        return self.request(
            f"/eventstore/{database_name}",
            method="GET",
            return_pandas_dataframe=False,
            headers=self.headers,
        )

In [3]:
radifar_cfg = ClientConfig(server_url="http://localhost:8000",
                        email="new_user@molar.org",
                        password="new_password",
                        database_name="rewoven_main")

radifar = RewovenClient(radifar_cfg)

radifar.database_creation_request(
    superuser_fullname="Muhammad Radifar",
    alembic_revisions=["rewoven_main@head"]
)

[21:15:57] INFO     Database request created                                    ]8;id=27589;file:///home/radifar/anaconda3/envs/molar-hackathon/lib/python3.9/site-packages/molar/client.py\client.py]8;;\:107

{'msg': 'Database request created'}

In [4]:
admin_client.get_alembic_revisions()

,revision,log_entry,branch_labels
0,0bc99b5f8fcc,Rev: 0bc99b5f8fcc (head)\nParent: f31c7d486f1f...,"[eventsourcing, compchem]"
1,311e0c050706,Rev: 311e0c050706 (head)\nParent: bf3c5d811155...,[molar-main]
2,86c6468a76a8,Rev: 86c6468a76a8 (head)\nParent: f31c7d486f1f...,"[rewoven_main, eventsourcing]"


In [5]:
admin_client.approve_database("rewoven_main")

[21:16:07] INFO     Database rewoven_main created.                              ]8;id=306018;file:///home/radifar/anaconda3/envs/molar-hackathon/lib/python3.9/site-packages/molar/client.py\client.py]8;;\:107

{'msg': 'Database rewoven_main created.'}

In [6]:
radifar.view_entries()

[]

In [7]:
dbinfo = radifar.get_database_information()

In [8]:
dbinfo[dbinfo.table_name == "step"]

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
34,step,created_on,timestamp without time zone,timestamp,NO,None,None,None
35,step,depends_on,uuid,uuid,YES,None,None,None
36,step,input,text,text,YES,None,None,None
37,step,log,text,text,YES,None,None,None
38,step,output,text,text,NO,None,None,None
39,step,run,text,text,YES,None,None,None
40,step,shell_command,text,text,YES,None,None,None
41,step,step_id,uuid,uuid,NO,step_pkey,None,step.step_id
42,step,updated_on,timestamp without time zone,timestamp,NO,None,None,None


In [9]:
dbinfo[dbinfo.table_name == "event_aggregates"]

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
9,event_aggregates,created_on,timestamp without time zone,timestamp,NO,None,None,None
10,event_aggregates,event_aggregates,jsonb,jsonb,NO,None,None,None
11,event_aggregates,event_aggregates_id,uuid,uuid,NO,event_aggregates_pkey,None,event_aggregates.event_aggregates_id
12,event_aggregates,tag_name,text,text,NO,None,None,None
13,event_aggregates,tag_parent,text,text,NO,None,None,None
14,event_aggregates,tagged_event,uuid,uuid,NO,None,None,None
15,event_aggregates,updated_on,timestamp without time zone,timestamp,NO,None,None,None


In [10]:
event = radifar.create_entry(type="software", data={"name": "Gromacs", "version": "2021.10"})
event

{'type': 'software',
 'id': 1,
 'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
 'event': 'create',
 'data': {'name': 'Gromacs', 'version': '2021.10'},
 'timestamp': '2021-10-24T14:16:23.542788',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [11]:
radifar.view_entries()

[{'type': 'software',
  'id': 1,
  'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
  'event': 'create',
  'data': {'name': 'Gromacs', 'version': '2021.10'},
  'timestamp': '2021-10-24T14:16:23.542788',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1}]

In [12]:
event = radifar.create_entry(
    type="event_aggregates", 
    data={"tag_name": "initial_tag",
          "tag_parent": "None",
          "tagged_event": event["uuid"],
          "event_aggregates": radifar.view_entries()
         })
event

{'type': 'event_aggregates',
 'id': 2,
 'uuid': '2c7563b3-7ca8-424f-971e-c0f34af64d48',
 'event': 'create',
 'data': {'tag_name': 'initial_tag',
  'tag_parent': 'None',
  'tagged_event': '1bd9af64-9e07-4508-9e93-01a605a07edd',
  'event_aggregates': [{'id': 1,
    'data': {'name': 'Gromacs', 'version': '2021.10'},
    'type': 'software',
    'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
    'event': 'create',
    'user_id': 1,
    'timestamp': '2021-10-24T14:16:23.542788',
    'alembic_version': ['86c6468a76a8']}]},
 'timestamp': '2021-10-24T14:16:37.947216',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [13]:
radifar.view_entries()

[{'type': 'software',
  'id': 1,
  'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
  'event': 'create',
  'data': {'name': 'Gromacs', 'version': '2021.10'},
  'timestamp': '2021-10-24T14:16:23.542788',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1},
 {'type': 'event_aggregates',
  'id': 2,
  'uuid': '2c7563b3-7ca8-424f-971e-c0f34af64d48',
  'event': 'create',
  'data': {'tag_name': 'initial_tag',
   'tag_parent': 'None',
   'tagged_event': '1bd9af64-9e07-4508-9e93-01a605a07edd',
   'event_aggregates': [{'id': 1,
     'data': {'name': 'Gromacs', 'version': '2021.10'},
     'type': 'software',
     'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
     'event': 'create',
     'user_id': 1,
     'timestamp': '2021-10-24T14:16:23.542788',
     'alembic_version': ['86c6468a76a8']}]},
  'timestamp': '2021-10-24T14:16:37.947216',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1}]

In [15]:
radifar.query_database(types="event_aggregates")

,event_aggregates_id,created_on,updated_on,tag_name,tag_parent,tagged_event,event_aggregates
0,2c7563b3-7ca8-424f-971e-c0f34af64d48,2021-10-24T14:16:37.947216,2021-10-24T14:16:37.947216,initial_tag,None,1bd9af64-9e07-4508-9e93-01a605a07edd,"[{'id': 1, 'data': {'name': 'Gromacs', 'versio..."


In [16]:
radifar.create_entry(
    type="step",
    data={
        "output": "ligands/IBP.pdb",
        "shell_command": "python 01_fetch_ligand.py IBP"
    }
    )

{'type': 'step',
 'id': 3,
 'uuid': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
 'event': 'create',
 'data': {'output': 'ligands/IBP.pdb',
  'shell_command': 'python 01_fetch_ligand.py IBP'},
 'timestamp': '2021-10-24T14:19:05.079747',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [17]:
event = radifar.create_entry(
    type="step",
    data={
        "input": "ligands/IBP.pdb",
        "output": "ligands/IBP.H.mol2",
        "shell_command": "python 02_add_hydrogens.py IBP",
        "depends_on": radifar.view_entries()[2]['uuid']
    })

event

{'type': 'step',
 'id': 4,
 'uuid': '39be99f9-cee4-4203-a8c5-ef9a91d282c8',
 'event': 'create',
 'data': {'input': 'ligands/IBP.pdb',
  'output': 'ligands/IBP.H.mol2',
  'depends_on': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
  'shell_command': 'python 02_add_hydrogens.py IBP'},
 'timestamp': '2021-10-24T14:19:06.812062',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [18]:
event = radifar.create_entry(
    type="step",
    data={
        "input": "ligands/IBP.H.mol2",
        "output": "ligands/IBP.H.min.pdb",
        "shell_command": "python 02_energy_minimize.py IBP",
        "depends_on": event['uuid']
    })

event

{'type': 'step',
 'id': 5,
 'uuid': 'f2e2731d-7976-4ae6-b572-c7398b69fdf5',
 'event': 'create',
 'data': {'input': 'ligands/IBP.H.mol2',
  'output': 'ligands/IBP.H.min.pdb',
  'depends_on': '39be99f9-cee4-4203-a8c5-ef9a91d282c8',
  'shell_command': 'python 02_energy_minimize.py IBP'},
 'timestamp': '2021-10-24T14:19:08.423345',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [19]:
event = radifar.create_entry(
    type="step",
    data={
        "input": "ligands/IBP.min.pdb",
        "output": "ligands/IBP.params.gro",
        "shell_command": "python 04_generate_parameter.py IBP",
        "depends_on": event['uuid']
    })

event

{'type': 'step',
 'id': 6,
 'uuid': 'dabac163-45ce-4933-bf7e-a66c2b622161',
 'event': 'create',
 'data': {'input': 'ligands/IBP.min.pdb',
  'output': 'ligands/IBP.params.gro',
  'depends_on': 'f2e2731d-7976-4ae6-b572-c7398b69fdf5',
  'shell_command': 'python 04_generate_parameter.py IBP'},
 'timestamp': '2021-10-24T14:19:10.049372',
 'alembic_version': ['86c6468a76a8'],
 'user_id': 1}

In [20]:
radifar.view_entries()

[{'type': 'software',
  'id': 1,
  'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
  'event': 'create',
  'data': {'name': 'Gromacs', 'version': '2021.10'},
  'timestamp': '2021-10-24T14:16:23.542788',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1},
 {'type': 'event_aggregates',
  'id': 2,
  'uuid': '2c7563b3-7ca8-424f-971e-c0f34af64d48',
  'event': 'create',
  'data': {'tag_name': 'initial_tag',
   'tag_parent': 'None',
   'tagged_event': '1bd9af64-9e07-4508-9e93-01a605a07edd',
   'event_aggregates': [{'id': 1,
     'data': {'name': 'Gromacs', 'version': '2021.10'},
     'type': 'software',
     'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
     'event': 'create',
     'user_id': 1,
     'timestamp': '2021-10-24T14:16:23.542788',
     'alembic_version': ['86c6468a76a8']}]},
  'timestamp': '2021-10-24T14:16:37.947216',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1},
 {'type': 'step',
  'id': 3,
  'uuid': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
  'event': 'create',
  '

In [21]:
entries = radifar.view_entries()
reverse_entries = entries[::-1]

last_tag = next(event for event in reverse_entries if event["type"] == "event_aggregates")

first_workflow_aggregates = entries[last_tag["id"]:]

event = radifar.create_entry(
    type="event_aggregates", 
    data={"tag_name": "first_workflow",
          "tag_parent": "initial_tag",
          "tagged_event": event["uuid"],
          "event_aggregates": first_workflow_aggregates
         })
event

{'type': 'event_aggregates',
 'id': 7,
 'uuid': 'cc54a254-9fff-4f48-91b8-943d623bd838',
 'event': 'create',
 'data': {'tag_name': 'first_workflow',
  'tag_parent': 'initial_tag',
  'tagged_event': 'dabac163-45ce-4933-bf7e-a66c2b622161',
  'event_aggregates': [{'id': 3,
    'data': {'output': 'ligands/IBP.pdb',
     'shell_command': 'python 01_fetch_ligand.py IBP'},
    'type': 'step',
    'uuid': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
    'event': 'create',
    'user_id': 1,
    'timestamp': '2021-10-24T14:19:05.079747',
    'alembic_version': ['86c6468a76a8']},
   {'id': 4,
    'data': {'input': 'ligands/IBP.pdb',
     'output': 'ligands/IBP.H.mol2',
     'depends_on': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
     'shell_command': 'python 02_add_hydrogens.py IBP'},
    'type': 'step',
    'uuid': '39be99f9-cee4-4203-a8c5-ef9a91d282c8',
    'event': 'create',
    'user_id': 1,
    'timestamp': '2021-10-24T14:19:06.812062',
    'alembic_version': ['86c6468a76a8']},
   {'id': 5,
    'dat

In [22]:
for entry in radifar.view_entries():
    if entry["type"] == "software":
        radifar.delete_entry(type="software", uuid=entry['uuid'])
    elif entry["type"] == "step":
        radifar.delete_entry(type="step", uuid=entry['uuid'])

radifar.view_entries()

[{'type': 'software',
  'id': 1,
  'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
  'event': 'create',
  'data': {'name': 'Gromacs', 'version': '2021.10'},
  'timestamp': '2021-10-24T14:16:23.542788',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1},
 {'type': 'event_aggregates',
  'id': 2,
  'uuid': '2c7563b3-7ca8-424f-971e-c0f34af64d48',
  'event': 'create',
  'data': {'tag_name': 'initial_tag',
   'tag_parent': 'None',
   'tagged_event': '1bd9af64-9e07-4508-9e93-01a605a07edd',
   'event_aggregates': [{'id': 1,
     'data': {'name': 'Gromacs', 'version': '2021.10'},
     'type': 'software',
     'uuid': '1bd9af64-9e07-4508-9e93-01a605a07edd',
     'event': 'create',
     'user_id': 1,
     'timestamp': '2021-10-24T14:16:23.542788',
     'alembic_version': ['86c6468a76a8']}]},
  'timestamp': '2021-10-24T14:16:37.947216',
  'alembic_version': ['86c6468a76a8'],
  'user_id': 1},
 {'type': 'step',
  'id': 3,
  'uuid': '372ec800-9468-4b16-a77f-ff41e5a0bec3',
  'event': 'create',
  '

In [24]:
radifar.query_database(types="software")

""


In [25]:
radifar.query_database(types="step")

""


In [49]:
def go_to_tag(name: str):
    tag = radifar.query_database(
        types="event_aggregates",
        filters={
            'type': "event_aggregates.tag_name",
            "op": "==",
            "value": name
    })
    
    for event in tag.event_aggregates[0]:
        if event["event"] == "create":
            radifar.create_entry(type=event["type"], data=event["data"])

In [50]:
go_to_tag("initial_tag")

In [51]:
radifar.query_database(types="software")

,software_id,created_on,updated_on,name,version
0,26e8b807-2c9c-4f51-b6ed-dfee87edf32c,2021-10-24T14:49:09.308424,2021-10-24T14:49:09.308424,Gromacs,2021.10


In [52]:
radifar.query_database(types="step")

""


In [53]:
go_to_tag("first_workflow")

In [54]:
radifar.query_database(types="software")

,software_id,created_on,updated_on,name,version
0,26e8b807-2c9c-4f51-b6ed-dfee87edf32c,2021-10-24T14:49:09.308424,2021-10-24T14:49:09.308424,Gromacs,2021.10


In [56]:
radifar.query_database(types="step")

,step_id,created_on,updated_on,output,shell_command,input,depends_on
0,9acb51bc-e44b-416f-a166-1bbcf8b1ca69,2021-10-24T14:50:15.988173,2021-10-24T14:50:15.988173,ligands/IBP.pdb,python 01_fetch_ligand.py IBP,None,None
1,9f639412-7acb-4510-b7af-56d6cfd311aa,2021-10-24T14:50:16.032053,2021-10-24T14:50:16.032053,ligands/IBP.H.mol2,python 02_add_hydrogens.py IBP,ligands/IBP.pdb,372ec800-9468-4b16-a77f-ff41e5a0bec3
2,9dd60131-bdaf-41e7-8740-1271a8537b55,2021-10-24T14:50:16.086936,2021-10-24T14:50:16.086936,ligands/IBP.H.min.pdb,python 02_energy_minimize.py IBP,ligands/IBP.H.mol2,39be99f9-cee4-4203-a8c5-ef9a91d282c8
3,651f22ba-96fb-4f2a-b4d6-20427672cdfc,2021-10-24T14:50:16.112338,2021-10-24T14:50:16.112338,ligands/IBP.params.gro,python 04_generate_parameter.py IBP,ligands/IBP.min.pdb,f2e2731d-7976-4ae6-b572-c7398b69fdf5
